In [2]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

print("Currently working at {}".format(os.getcwd()))

Currently working at /Users/gieunkwak/Data Analytics/random_things/tridge_demo


In [3]:
import mysql.connector
from mysql.connector import Error
from pandas.io import sql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import time

import investpy
from fredapi import Fred
import json
from matplotlib import pyplot as plt
import quandl
from datetime import date
from app.utils.fetch_data import *
from app.utils.db_connection import *

from collections import Counter

In [29]:
options = fred.search('Producer Price Index by Commodity: Farm Products').reset_index()
options['food_name'] = options['title'].str.split(": ").str[-1]
options['observation_start_year'] = options['observation_start'].dt.year
# filter out unwanted indexes
options = options.loc[~options['food_name'].str.contains('DISCONTINUED')]
options = options.loc[options['frequency_short']=='M']
options = options.loc[options['seasonal_adjustment_short']=='NSA']

In [32]:
options.columns

Index(['series id', 'id', 'realtime_start', 'realtime_end', 'title',
       'observation_start', 'observation_end', 'frequency', 'frequency_short',
       'units', 'units_short', 'seasonal_adjustment',
       'seasonal_adjustment_short', 'last_updated', 'popularity', 'notes',
       'food_name', 'observation_start_year'],
      dtype='object')

In [53]:
latest = options.groupby('food_name').agg({"observation_start_year":"min"}).reset_index()
latest = latest.merge(options, on=['food_name', 'observation_start_year'], how='left')

In [54]:
popular = latest.groupby('food_name').agg({"popularity":"max"}).reset_index()
popular = popular.merge(latest, on=['food_name', 'popularity'], how='left')

In [55]:
final = popular.drop_duplicates(subset=['food_name'])

In [57]:
final.shape

(136, 18)

In [51]:
for k,v in Counter(final.food_name).items():
    if v>1:
        print(k)

Rough Rice


In [52]:
final.loc[final.food_name=='Rough Rice']

,food_name,popularity,observation_start_year,series id,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes
98,Rough Rice,1,1991,WPU012301,WPU012301,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1991-12-01,2022-04-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Not Seasonally Adjusted,NSA,2022-05-12 08:05:15-05:00,None
99,Rough Rice,1,1991,WPU0123,WPU0123,2022-05-16,2022-05-16,Producer Price Index by Commodity: Farm Produc...,1991-12-01,2022-04-01,Monthly,M,Index Dec 1991=100,Index Dec 1991=100,Not Seasonally Adjusted,NSA,2022-05-12 08:05:40-05:00,None


In [39]:
df.shape

(140, 18)

In [40]:
latest.shape

(136, 2)

In [4]:
with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)

In [ ]:
today = date.today().strftime("%Y-%m-%d")
today

df = fred_fred('WPU01190102', observation_start='2000-01-01', observation_end=today)